In [70]:
import os
import win32com.client as win32
import time
import pandas as pd

aspen = win32.Dispatch("Apwn.Document")
aspen.InitFromArchive2(os.path.abspath("D:/saf_hdo/250924_pyrolysis_oil_CC_1.apw"))
aspen.Visible = True
aspen.SuppressDialogs = True


In [71]:
# aspen.Engine.Run2()
rxtor1 = aspen.Tree.FindNode("\Data\Blocks\R-201\Input\CONV")
rxtor2 = aspen.Tree.FindNode("\Data\Blocks\R-202\Input\CONV")
rxtor3 = aspen.Tree.FindNode("\Data\Blocks\R-203\Input\CONV")

hdo_prod = aspen.Tree.FindNode("\Data\Streams\\208\Output\MASSFLOW\MIXED")

In [72]:
hdo_WATER = hdo_prod.Elements("WATER")
hdo_C21 = hdo_prod.Elements("C21H24O5")

print("  RXN 30 || Produced WATER || Produced C21H24O5")
print(f"{rxtor1.Elements('30').Value} || {hdo_WATER.Value} || {hdo_C21.Value}")

rxtor1.Elements("30").Value = 0.1
time.sleep(2)
aspen.Engine.Run2()
time.sleep(2)

print(f"{rxtor1.Elements('30').Value} || {hdo_WATER.Value} || {hdo_C21.Value}")

  RXN 30 || Produced WATER || Produced C21H24O5
1.0 || 14124.4453 || 0.0
0.1 || 13724.0476 || 1584.31258


In [73]:
data_path = "D:\saf_hdo\HDO_exp.xlsx"
xls = pd.ExcelFile(data_path)
data = pd.read_excel(xls).fillna(0)

In [74]:
component_list = [
    'BIOMASS', 'WATER', 'H2', 'O2', 'N2', 'AR', 'CO', 'CO2', 'CH4',
    'C2H4', 'C2H4O', 'C2H4O2', 'C2H6', 'C2H6O', 'C3H6', 'C3H8',
    'C3H8O', 'C3H8O2', 'C4H10', 'C4H4O', 'C4H8O', 'C5H4O2', 'C5H12',
    'C6H6', 'C6H6O', 'C6H8O', 'C6H12', 'C6H12O6', 'C6H14', 'C6H14O6',
    'C7H8', 'C7H8O', 'C7H14', 'C7H14-CY', 'C7H14-ME', 'C8H10', 'C8H10O',
    'C8H16', 'C8H18', 'C8H16-CY', 'C9H10O3', 'C9H18', 'C9H20', 'C9H20-A',
    'C10H22-C', 'C11H14O', 'C11H20-C', 'C11H24', 'C12H16O2', 'C12H20-C',
    'C12H26', 'C13H18O2', 'C13H20O2', 'C13H26', 'C13H28', 'C14H12',
    'C14H20O2', 'C14H28A', 'C14H30', 'C15H28', 'C16H32', 'C16H34',
    'C17H34', 'C17H36', 'C18H38', 'C19H24O2', 'C19H38', 'C19H40', 'C20H40',
    'C20H42', 'C21H24O5', 'C21H42', 'C22H26O5', 'C22H44', 'C23H28O5',
    'C23H46', 'C24H32O3', 'C24H48', 'C25H30O3', 'C25H50', 'C26H42O4',
    'C26H52', 'C27H54', 'C28H56', 'C29H58', 'C30H62', 'SO2', 'NO2',
    'ACIDS', 'ALDEHYDE', 'KETONES', 'ALCOHOLS', 'GUAIACOL', 'LMWS',
    'HMWS', 'EXTRACTI', 'N2COMP', 'SCOMPOUN', 'S', 'C', 'ASH', 'SIO2',
    'LMWLA', 'LMWLB', 'HLB', 'NH3'
]
component_to_carbon_number = {
    "ACIDS": 4,
    "ALDEHYDE": 8,
    "KETONES": 3,
    "ALCOHOLS": 6,
    "GUAIACOL": 7,
    "LMWS": 6,
    "HMWS": 12,
    "EXTRACTI": 20,
    # "N2COMP": 8,  # TODO: do we count them?
    # "SCOMPOUN": 12,
    "LMWLA": 16,
    "LMWLB": 12,
    "HLB": 17,
}
carbon_number_list = data['Carbon'].tolist()
carbon_number_to_component = {n: [c for c in component_list if f"C{n}" in c] for n in carbon_number_list}
for c, n in component_to_carbon_number.items():
    if n in carbon_number_list:
        carbon_number_to_component[n].append(c)

hdo_rxtors = [rxtor1, rxtor2, rxtor3]
n_rxns = [len(r.Elements) for r in hdo_rxtors]

hdo_prod_composition = [hdo_prod.Elements(c).Value for c in component_list]

# node = aspen.Tree.FindNode(f"\Data\Streams\\208\Output\MASSFLOW\MIXED")
# massflow = [0 if node.Elements(c).Value is None else node.Elements(c).Value for c in component_list]
# total_massflow = sum(massflow)
# carbon_number_composition = {n: sum([massflow[component_list.index(c)]/total_massflow for c in cs]) for n, cs in carbon_number_to_component.items()}


In [75]:
def get_carbon_number_composition(stream_no):
    node = aspen.Tree.FindNode(f"\Data\Streams\\{stream_no}\Output\MASSFLOW\MIXED")
    massflow = [0 if node.Elements(c).Value is None else node.Elements(c).Value for c in component_list]
    total_massflow = sum(massflow)
    carbon_number_composition = {n: sum([massflow[component_list.index(c)]/total_massflow for c in cs]) for n, cs in carbon_number_to_component.items()}
    return carbon_number_composition

get_carbon_number_composition(208)

{6: 0.008563108717343856,
 7: 0.010174536360311846,
 8: 0.015492652320538054,
 9: 0.03374726898480269,
 10: 0.021976491703757588,
 11: 0.02340851629752672,
 12: 0.02097552111615141,
 13: 0.029146371762567912,
 14: 0.009941343016103612,
 15: 0.012137583920344626,
 16: 0.005601037522565215,
 17: 0.0031051141264400606,
 18: 0.0029491868117174965,
 19: 0.0014447330485379778,
 20: 0.0009121204769747764,
 21: 0.04093845935414537,
 22: 0.0,
 23: 0.027531712366172897,
 24: 0.027531711590976265,
 25: 0.21905666483368488,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0}

In [77]:
case_target = 'a'
target_sum = sum(data[case_target])
target_data = {n: c / target_sum for n, c in zip(data['Carbon'], data[case_target])}

r_now = []
for ii, rr in enumerate(hdo_rxtors):
    for n in range(n_rxns[ii]):
        r_now.append(rr.Elements(f"{n+1}").Value)


def get_hdo_results(rxn_coef):
    # print(rxn_coef)
    for n in range(n_rxns[0]):
        rxtor1.Elements(f"{n+1}").Value = rxn_coef[n]

    time.sleep(2)
    aspen.Engine.Run2()
    time.sleep(2)

    prod = get_carbon_number_composition(208)
    mse = sum((target_data[n] - prod[n])**2 for n in target_data.keys())
    print(f"Success: {mse}")
    return mse * 10


r_new = r_now[:n_rxns[0]].copy()
r_new[29] = 1.0
print(get_hdo_results(r_new))

[1.0, 1.0, 1.0, 1.0, 1.0, 0.03, 0.85, 1e-05, 0.015, 1.0, 1.0, 0.05, 0.38, 1.0, 0.44, 0.47, 1.0, 0.35, 0.25, 0.2, 1.0, 0.2, 1.0, 0.75, 0.4, 0.73, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Success: 0.1082820941483201
0.1082820941483201


In [83]:
from scipy.optimize import minimize
bounds = [(0, 1) for _ in range(n_rxns[0])]
r_init = r_now[:n_rxns[0]].copy()

def sum_coef(coef):
    print(coef)
    print(f"SUM: {sum(coef)}")
    return sum(coef)


result = minimize(
    fun=get_hdo_results,
    x0=r_init,
    bounds=bounds,
    method="SLSQP",
    options={"disp": True,
             'maxiter': 10},
)

if result.success:
    print("Optimization successful!")
    print(f"Optimal reaction coefficients: {result.x}")
    print(f"Minimum MSE: {result.fun}")
else:
    print("Optimization failed.")
    print(result.message)

[1.0e+00 1.0e+00 1.0e+00 1.0e+00 1.0e+00 3.0e-02 8.5e-01 1.0e-05 1.5e-02
 1.0e+00 1.0e+00 5.0e-02 3.8e-01 1.0e+00 4.4e-01 4.7e-01 1.0e+00 3.5e-01
 2.5e-01 2.0e-01 1.0e+00 2.0e-01 1.0e+00 7.5e-01 4.0e-01 7.3e-01 1.0e+00
 1.0e+00 0.0e+00 1.0e+00 1.0e+00 1.0e+00 1.0e+00 1.0e+00 1.0e+00]
SUM: 24.115009999999998
[9.99999985e-01 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 3.00000000e-02 8.50000000e-01 1.00000000e-05
 1.50000000e-02 1.00000000e+00 1.00000000e+00 5.00000000e-02
 3.80000000e-01 1.00000000e+00 4.40000000e-01 4.70000000e-01
 1.00000000e+00 3.50000000e-01 2.50000000e-01 2.00000000e-01
 1.00000000e+00 2.00000000e-01 1.00000000e+00 7.50000000e-01
 4.00000000e-01 7.30000000e-01 1.00000000e+00 1.00000000e+00
 0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00]
SUM: 24.115009985098837
[1.00000000e+00 9.99999985e-01 1.00000000e+00 1.00000000e+00
 1.00000000e+00 3.00000000e-02 8.50000000e-01 1.00000000e-05
 1.5000000

In [57]:
# Close the file without saving
# SaveOption: 0 = Do not save changes
#             1 = Save changes
#             2 = Prompt the user
aspen.Close(0)